# Handwritten Digit Recognization

### import the Libraries

In [22]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


### Preprocessing the data

In [23]:
num_classes=10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train/255
x_test = x_test/255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Creating the model

In [24]:
batch_size = 128
num_classes = 10
epochs = 10
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])


In [25]:
model.save('mnist_model.keras')

### Train the model

In [14]:
hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
print("The model has successfully trained")

model.save('mnist_model.keras')

print("Model saved as mnist_model.keras")

Epoch 1/10
469/469 [==============================] - 93s 198ms/step - loss: 2.2444 - accuracy: 0.2248 - val_loss: 2.1878 - val_accuracy: 0.5111
Epoch 2/10
469/469 [==============================] - 107s 228ms/step - loss: 2.1539 - accuracy: 0.3737 - val_loss: 2.0724 - val_accuracy: 0.6495
Epoch 3/10
469/469 [==============================] - 106s 225ms/step - loss: 2.0301 - accuracy: 0.4893 - val_loss: 1.9093 - val_accuracy: 0.7257
Epoch 4/10
469/469 [==============================] - 104s 221ms/step - loss: 1.8565 - accuracy: 0.5756 - val_loss: 1.6875 - val_accuracy: 0.7566
Epoch 5/10
469/469 [==============================] - 111s 237ms/step - loss: 1.6398 - accuracy: 0.6260 - val_loss: 1.4279 - val_accuracy: 0.7783
Epoch 6/10
469/469 [==============================] - 102s 218ms/step - loss: 1.4106 - accuracy: 0.6649 - val_loss: 1.1757 - val_accuracy: 0.7973
Epoch 7/10
469/469 [==============================] - 104s 223ms/step - loss: 1.2114 - accuracy: 0.6916 - val_loss: 0.9712 - 

### Evaluating the Model

In [26]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.3012683391571045
Test accuracy: 0.11940000206232071


### Creating GUI to Predict the Digits

In [1]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

### Testing the model

In [2]:
# Load your model
model = load_model('mnist.h5')

def predict_digit(img):
    img = img.resize((28,28))
    img = img.convert('L')
    img = np.array(img)
    img = img.reshape(1,28,28,1)
    img = img/255.0
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        #self.title("Handwriting Recognition App")
        self.canvas = tk.Canvas(self, width=400, height=400, bg="Blue", cursor="cross")
        self.label = tk.Label(self, text="Thinking....", font=("Helvetica", 50))
        self.classify_btn = tk.Button(self, text="Recognize", command =       self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command = self.clear_all)
        
        self.canvas.grid(row=0, column=0, pady=2, sticky='w', )
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()
        rect = win32gui.GetWindowRect(HWND)
        im = ImageGrab.grab(rect)
        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def clear_all(self):
        self.canvas.delete("all")

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

1/1 [==============================] - 0s 415ms/step


In [6]:
from keras.datasets import mnist
import numpy as np

# Load MNIST dataset
(_, _), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the test data
x_test = x_test.astype('float32') / 255.0
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Load your trained model
model = load_model('mnist_model.keras')

# Make predictions
predictions = model.predict(x_test)

# Display the predicted digits
for i in range(10):  # Display the first 10 predictions
    predicted_digit = np.argmax(predictions[i])
    actual_digit = y_test[i]
    print(f"Actual Digit: {actual_digit}, Predicted Digit: {predicted_digit}")


313/313 [==============================] - 4s 13ms/step
Actual Digit: 7, Predicted Digit: 8
Actual Digit: 2, Predicted Digit: 4
Actual Digit: 1, Predicted Digit: 9
Actual Digit: 0, Predicted Digit: 4
Actual Digit: 4, Predicted Digit: 6
Actual Digit: 1, Predicted Digit: 7
Actual Digit: 4, Predicted Digit: 4
Actual Digit: 9, Predicted Digit: 6
Actual Digit: 5, Predicted Digit: 6
Actual Digit: 9, Predicted Digit: 7


In [3]:
import tkinter as tk
from PIL import ImageGrab, Image
import numpy as np
from keras.models import load_model

# Load your model
model = load_model('mnist_model.keras')  # Replace with the correct model filename

def predict_digit(img):
    img = img.resize((28, 28))
    img = img.convert('L')
    img = np.array(img)
    img = img.reshape(1, 28, 28, 1)
    img = img / 255.0
    res = model.predict(img)
    return np.argmax(res), max(res[0])

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.x = self.y = 0
        self.title("Handwritten Digit Recognition App")
        self.canvas = tk.Canvas(self, width=400, height=400, bg="Blue", cursor="cross")
        self.label = tk.Label(self, text="Thinking....", font=("Helvetica", 50))
        self.classify_btn = tk.Button(self, text="Recognize", command=self.classify_handwriting)
        self.button_clear = tk.Button(self, text="Clear", command=self.clear_all)
        
        self.canvas.grid(row=0, column=0, pady=2, sticky='w')
        self.label.grid(row=0, column=1, pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        self.canvas.bind("<ButtonRelease-1>", self.classify_handwriting)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def classify_handwriting(self, event=None):
        x = self.winfo_rootx() + self.canvas.winfo_x()
        y = self.winfo_rooty() + self.canvas.winfo_y()
        x1 = x + self.canvas.winfo_width()
        y1 = y + self.canvas.winfo_height()
        im = ImageGrab.grab((x, y, x1, y1))
        digit, acc = predict_digit(im)
        self.label.configure(text=f"Predicted Digit: {digit}, Confidence: {int(acc*100)}%")

    def clear_all(self):
        self.canvas.delete("all")
        self.label.configure(text="Draw a digit and click 'Recognize'")

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 8
        self.canvas.create_oval(self.x - r, self.y - r, self.x + r, self.y + r, fill='black')

app = App()
app.mainloop()

from keras.datasets import mnist
import numpy as np

# Load MNIST dataset
(_, _), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the test data
x_test = x_test.astype('float32') / 255.0
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# Load your trained model
model = load_model('mnist_model.keras')

# Make predictions
predictions = model.predict(x_test)

# Display the predicted digits
for i in range(10):  # Display the first 10 predictions
    predicted_digit = np.argmax(predictions[i])
    actual_digit = y_test[i]
    print(f"Actual Digit: {actual_digit}, Predicted Digit: {predicted_digit}")



1/1 [==============================] - 0s 28ms/step


313/313 [==============================] - 3s 10ms/step
Actual Digit: 7, Predicted Digit: 8
Actual Digit: 2, Predicted Digit: 4
Actual Digit: 1, Predicted Digit: 9
Actual Digit: 0, Predicted Digit: 4
Actual Digit: 4, Predicted Digit: 6
Actual Digit: 1, Predicted Digit: 7
Actual Digit: 4, Predicted Digit: 4
Actual Digit: 9, Predicted Digit: 6
Actual Digit: 5, Predicted Digit: 6
Actual Digit: 9, Predicted Digit: 7
